<a href="https://colab.research.google.com/github/mojosolo/mojo-eleventy/blob/main/Agent_Swarms_Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain BeautifulSoup4 yfinance
!pip install openai
# !pip install openai==0.28
!pip install duckduckgo-search

In [ ]:
import os

In [ ]:
os.environ["OPENAI_API_KEY"] = ""
os.environ["SERPER_API_KEY"] = ""
os.environ["SEARCHAPI_API_KEY"] = ""
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]= ""


In [ ]:
# # Needed if you would like to display images in the notebook
# !pip install opencv-python scikit-image

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, AgentType, load_tools, Tool
from langchain.tools.yahoo_finance_news import YahooFinanceNewsTool
from langchain.tools import  DuckDuckGoSearchResults
from langchain.utilities.dalle_image_generator import DallEAPIWrapper
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [ ]:
llm = ChatOpenAI(model = "gpt-4", temperature=0.1)

In [ ]:

# One way to load tools...

# tools = [
#     Tool(
#         name="Intermediate Answer",
#         func=search.run,
#         description="useful for when you need to ask with search"
#     )

#     Tool(
#         name="",
#         func=search.run,
#         description="useful for when you need to ask with search"
#     )

#     )
# ]



In [ ]:
# New way to load tools
tools = load_tools(["searchapi", 'dalle-image-generator'], llm=llm)


agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    early_stopping_method='generate' #does one FINAL pass through the LLM to generate an output. or you can use "force" which just returns a constant string

)

In [ ]:
output = agent.run("1. what is the price of bitcoin. 2. summarize any recent news about Romania. 3. a photo of a bitcoin with a rocket flying by on the way to the moon. 4. whats the temperature in Switzerland ")



> Entering new AgentExecutor chain...
I need to gather information about the price of BTC, recent news about Romania, generate an image of a bitcoin with a rocket, and find the temperature in Switzerland. I should use the searchapi for the price of BTC and recent news about Romania, and use the Dall-E-Image-Generator for the image of a bitcoin with a rocket. For the temperature in Switzerland, I can use the searchapi as well.
Action: searchapi
Action Input: "price of BTC"
Observation: The Bitcoin price is $36,801.08, a change of 4.28% over the past 24 hours as of 12:00 a.m. The recent price action in Bitcoin Read more...
November 11, 2023 - The current price of Bitcoin is $36662.67 per (BTC / USD). Bitcoin is 46.70% below the all time high of $68789.63.
The live Bitcoin price today is $36978.52 USD with a 24-hour trading volume of $20904549889.67 USD. We update our BTC to USD price in real-time.
Check out the current Bitcoin (BTC) price, market cap, historical volatility, and buy Bit

In [ ]:
output

"1. The price of BTC is $36,801.08.\n2. Recent news about Romania includes NATO member Romania finding more drone fragments on its soil after Russia again hits southern Ukraine, Ukraine's Zelenskyy visiting neighboring Romania to discuss security and boost ties, and Tonga ending Rugby World Cup with a win against Romania.\n3. Here is a photo of a bitcoin with a rocket flying by on the way to the moon: [image link]\n4. The temperature in Switzerland varies depending on the location and time of year, but the average temperature in January is slightly above freezing, and in July and August, it is around 19/20 °C (66/68 °F)."

In [ ]:

agent2 = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    early_stopping_method='generate'

)
agent2.run(output)

#Lets try to do it another way

In [ ]:
from operator import itemgetter

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser

prompt1 = ChatPromptTemplate.from_template("what is the city {person} is from?")
prompt2 = ChatPromptTemplate.from_template(
    "what country is the city {city} in? respond in {language}"
)

model = ChatOpenAI()

chain1 = prompt1 | model | StrOutputParser()

chain2 = (
    {"city": chain1, "language": itemgetter("language")}
    | prompt2
    | model
    | StrOutputParser()
)

chain2.invoke({"person": "Trump", "language": "French"})

"Le pays d'origine de Donald Trump, le 45e président des États-Unis, est les États-Unis d'Amérique."

#Now lets try something more interesting..

In [ ]:
from langchain.schema.runnable import RunnableMap, RunnablePassthrough

# Adjusted prompt templates for digital content creation
prompt1 = ChatPromptTemplate.from_template(
    "Design a {attribute} color palette for a digital artwork. Return the hexadecimal codes of the colors and nothing else:"
)
prompt2 = ChatPromptTemplate.from_template(
    "What is a character or mascot that could represent a brand with the primary color: {color}? Return a brief character description and nothing else:"
)
prompt3 = ChatPromptTemplate.from_template(
    "Create a concept for a logo that incorporates the color: {color}. Describe the concept briefly and nothing else:"
)
prompt4 = ChatPromptTemplate.from_template(
    "Draft a brief for a digital advertisement campaign featuring {character} and the logo concept for {brand}."
)

# Assuming 'model' is a callable model object and 'StrOutputParser' is defined elsewhere
model_parser = model | StrOutputParser()

# Color palette generator based on an attribute
color_palette_generator = (
    {"attribute": RunnablePassthrough()} | prompt1 | {"color": model_parser}
)

# Character or mascot generator based on a color
character_generator = (
    color_palette_generator | {"color": RunnablePassthrough()} | prompt2 | {"character": model_parser}
)

# Logo concept generator based on a color
logo_concept_generator = (
    color_palette_generator | {"color": RunnablePassthrough()} | prompt3 | {"brand": model_parser}
)

# Digital advertisement campaign brief generator
campaign_brief_generator = (
    {"character": character_generator, "brand": logo_concept_generator} | prompt4
)


In [ ]:
prompt = campaign_brief_generator.invoke("Blue")
model.invoke(prompt)

AIMessage(content='Title: "Dive into Fun with Splash: A Digital Advertisement Campaign"\n\nObjective:\nThe objective of this digital advertisement campaign is to promote water-related activities and products through the friendly and energetic character of Splash, a sleek and vibrant blue dolphin. The campaign aims to create brand awareness and engage the target audience by evoking a sense of excitement, adventure, and connection to water-based experiences.\n\nTarget Audience:\nThe target audience for this campaign includes individuals of all ages who enjoy water-related activities such as swimming, snorkeling, diving, water sports, and beach vacations. Additionally, families, adventure enthusiasts, and nature lovers who value outdoor activities and have an affinity for aquatic-themed experiences are also part of the target audience.\n\nKey Message:\n"Experience the Joy of Water: Dive into Fun with Splash!" \n\nCampaign Elements:\n1. Digital Advertisements:\n   - Banner Ads: Display vis

#Merging Multiple Streams


#Oooh...Things are getting spicey!

In [ ]:
planner = (
    ChatPromptTemplate.from_template("Generate an argument about: {input}")
    | ChatOpenAI()
    | StrOutputParser()
    | {"base_response": RunnablePassthrough()}
)

arguments_for = (
    ChatPromptTemplate.from_template(
        "List the pros or positive aspects of {base_response}"
    )
    | ChatOpenAI()
    | StrOutputParser()
)
arguments_against = (
    ChatPromptTemplate.from_template(
        "List the cons or negative aspects of {base_response}"
    )
    | ChatOpenAI()
    | StrOutputParser()
)

final_responder = (
    ChatPromptTemplate.from_messages(
        [
            ("ai", "{original_response}"),
            ("human", "Pros:\n{results_1}\n\nCons:\n{results_2}"),
            ("system", "Generate a final response given the critique"),
        ]
    )
    | ChatOpenAI()
    | StrOutputParser()
)

chain = (
    planner
    | {
        "results_1": arguments_for,
        "results_2": arguments_against,
        "original_response": itemgetter("base_response"),
    }
    | final_responder
)

In [ ]:
chain.invoke({"input": "Bitcoin"})

"While Bitcoin has the potential to revolutionize the financial industry and become the future of currency, it is crucial to acknowledge the cons and potential challenges associated with it. The volatility of Bitcoin's value, lack of regulation, scalability issues, energy consumption, limited acceptance, lack of user-friendly infrastructure, and potential for market manipulation are valid concerns that need to be addressed.\n\nVolatility remains a significant obstacle for Bitcoin to be widely accepted as a stable currency. The fluctuating value of Bitcoin can make it risky for investors and businesses, and it may hinder its use as a medium of exchange. Additionally, the lack of regulation raises concerns about fraud and illegal activities, and the absence of consumer protection and dispute resolution mechanisms may deter widespread adoption.\n\nScalability limitations pose a challenge for Bitcoin's widespread adoption. The current design of the blockchain limits the number of transacti